In [1]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import random
from task_1c_v2 import cvr_op_scheme, cvr_tp_scheme

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] = True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4


In [2]:
# Load data
with open('Data/ALL_scenarios.json') as f:
    all_scenarios = json.load(f)

# Constants
OMEGA = 250  # Number of scenarios to sample
PI = 1 / OMEGA  # Probability of each sampled scenario - assumed to be equal
S = len(all_scenarios.keys()) - 1  # Total number of scenarios
T = 24  # Number of hours
WIND_CAPACITY = 200  # MWh

random.seed(123)

# Sample scenarios without replacement
in_sample_scenarios = random.sample(range(S), 250)

scenarios = {}

# Extract sampled scenarios from dictionary containing all scenarios
for i in range(len(in_sample_scenarios)):
    scenarios[str(i)] = all_scenarios[str(in_sample_scenarios[i])]
    scenarios[str(i)]['Original Index'] = in_sample_scenarios[i]

# Indices of all scenarios
all_indices = list(range(S))



In [3]:
alpha = 0.9
beta_values = np.arange(0,1 + 0.1, 0.1)

In [4]:

# Initialize lists to store results
out_sample_profits = []
expected_values = []

num_scenarios = [170, 250, 360, 430, 540]

for OMEGA in num_scenarios:
    in_sample_scenarios = random.sample(range(len(all_scenarios.keys()) - 1), OMEGA)
    
    scenarios = {}
    for i, idx in enumerate(in_sample_scenarios):
        scenarios[str(i)] = all_scenarios[str(idx)]
        scenarios[str(i)]['Original Index'] = idx
    
    # Indices of out-of-sample scenarios
    out_of_sample_indices = list(set(all_indices) - set(in_sample_scenarios))

    # Sample 950 out-of-sample scenarios without replacement
    out_sample_indices = random.sample(out_of_sample_indices, 950)
    print(out_sample_indices)

    # Extract out-of-sample scenarios from the dictionary containing all scenarios
    out_sample_scenarios = {}
    for i, idx in enumerate(out_sample_indices):
        out_sample_scenarios[str(i)] = all_scenarios[str(idx)]
        out_sample_scenarios[str(i)]['Original Index'] = idx

    # Run optimization for one-price
    results_per_beta_op, p_DA_values_per_beta_op = cvr_op_scheme(scenarios, WIND_CAPACITY, T, OMEGA, alpha, beta_values, minimize_printouts=False, mip_gap = 1e-4) 
    beta_val = 0.5
    optimal_DA_oneprice = np.array(p_DA_values_per_beta_op[beta_val])
    expected_profit = results_per_beta_op[beta_val]['Expected Profit']

    out_sample_earnings = 0

    for scenario_idx in out_sample_scenarios:
        scenario = out_sample_scenarios[scenario_idx]

        x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_oneprice

        realized_imbalance = np.array(scenario['Wind Power [MW]']) - optimal_DA_oneprice

        price_coefficient = 1.2 * np.array(scenario['System Balance State']) + 0.9 * (1 - np.array(scenario['System Balance State']))
        y = realized_imbalance * price_coefficient * scenario['Spot Price [EUR/MWh]']

        payment_earnings = x + y
        
        out_sample_earnings += np.sum(payment_earnings)

        
    # Calculate average payment or earnings over out-of-sample scenarios
    average_out_sample_profits = out_sample_earnings / len(out_sample_scenarios)
    out_sample_profits.append(average_out_sample_profits)

    expected_values.append(expected_profit)
        
    print(f"Number of scenarios {OMEGA}  in-sample profits: {expected_profit}, out-sample profits: {average_out_sample_profits}")


[42, 955, 1472, 1110, 1632, 292, 1069, 1202, 91, 1052, 1216, 311, 726, 1947, 828, 487, 1338, 1932, 68, 1038, 1127, 1345, 1726, 638, 1570, 1301, 29, 715, 1623, 864, 503, 1531, 137, 1950, 562, 630, 368, 928, 801, 948, 92, 1357, 27, 1111, 1501, 361, 436, 100, 596, 1178, 363, 1701, 1638, 1192, 1874, 428, 672, 1118, 1046, 494, 625, 1185, 202, 227, 985, 1369, 1223, 1253, 619, 1799, 430, 1166, 148, 1418, 876, 906, 1778, 1765, 999, 243, 132, 1279, 965, 1694, 1458, 971, 1159, 799, 338, 1104, 297, 429, 467, 533, 79, 539, 458, 160, 900, 54, 23, 920, 1463, 718, 1227, 64, 89, 546, 1626, 1905, 1555, 643, 410, 674, 246, 1981, 1495, 1434, 1833, 724, 468, 1154, 55, 1666, 809, 1750, 336, 1043, 495, 924, 489, 1142, 1476, 969, 873, 1259, 417, 871, 168, 1951, 1991, 634, 307, 1065, 1627, 1602, 234, 279, 268, 1028, 790, 1240, 1373, 37, 990, 1920, 1828, 1975, 812, 470, 872, 101, 232, 1137, 1675, 208, 1328, 1480, 1864, 1637, 182, 1739, 1490, 453, 859, 395, 934, 1832, 1064, 309, 10, 355, 1758, 1770, 189, 737, 6

In [8]:
print(f"out-sample {out_sample_profits}")
print(f"In-sample {expected_values}")


out-sample [184981.3328431357, 187500.30520992115, 176700.7872865299, 184144.01134708532, 174809.90757300975]
In-sample [123940.40390379727, 172498.65717004443, 261501.58585270707, 304605.3441613093, 409068.868752726]


In [4]:
out_sample_profits_tp = []
expected_values_tp = []

num_scenarios = [170, 250, 360, 430, 540]

# Sample in-sample scenarios without replacement
for OMEGA in num_scenarios:
    in_sample_scenarios = random.sample(range(len(all_scenarios.keys()) - 1), OMEGA)
    
    scenarios = {}
    for i, idx in enumerate(in_sample_scenarios):
        scenarios[str(i)] = all_scenarios[str(idx)]
        scenarios[str(i)]['Original Index'] = idx
    
    # Indices of out-of-sample scenarios
    out_of_sample_indices = list(set(all_indices) - set(in_sample_scenarios))

    # Sample 950 out-of-sample scenarios without replacement
    out_sample_indices = random.sample(out_of_sample_indices, 950)

    # Extract out-of-sample scenarios from the dictionary containing all scenarios
    out_sample_scenarios = {}
    for i, idx in enumerate(out_sample_indices):
        out_sample_scenarios[str(i)] = all_scenarios[str(idx)]
        out_sample_scenarios[str(i)]['Original Index'] = idx

    # Run optimization for one-price
    results_per_beta_tp, p_DA_values_per_beta_tp = cvr_tp_scheme(scenarios, WIND_CAPACITY, T, OMEGA, alpha, beta_values, minimize_printouts=False, mip_gap = 1e-4) 
    beta_val = 0.5
    optimal_DA_twoprice = np.array(p_DA_values_per_beta_tp[beta_val])
    expected_profit = results_per_beta_tp[beta_val]['Expected Profit']

    out_sample_earnings_tp = 0

    for scenario_idx in out_sample_scenarios:
        scenario = out_sample_scenarios[scenario_idx]

        x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_twoprice

        realized_imbalance = np.array(scenario['Wind Power [MW]']) - optimal_DA_twoprice

        price_coefficients = np.zeros(T)

        for t in range(T):
            if (realized_imbalance[t] > 0) and (scenario['System Balance State'][t] == 0):
                price_coefficients[t] = 0.9
            elif (realized_imbalance[t] > 0) and (scenario['System Balance State'][t] == 1):
                price_coefficients[t] = 1.0
            elif (realized_imbalance[t] < 0) and (scenario['System Balance State'][t] == 0):
                price_coefficients[t] = -1.0
            elif (realized_imbalance[t] < 0) and (scenario['System Balance State'][t] == 1):
                price_coefficients[t] = -1.2

        y = np.abs(realized_imbalance) * price_coefficients * scenario['Spot Price [EUR/MWh]']

        payment_earnings_tp = x + y

        out_sample_earnings_tp += np.sum(payment_earnings_tp)

    # Calculate average payment or earnings over out-of-sample scenarios
    average_out_sample_profits_tp = out_sample_earnings_tp / len(out_sample_scenarios)
    out_sample_profits_tp.append(average_out_sample_profits_tp)

    expected_values_tp.append(expected_profit)
        
    print(f"Number of scenarios {OMEGA}  in-sample profits function: {expected_profit}, out-sample profits: {average_out_sample_profits_tp}")

Solving for beta = 0.00...
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-26
-----------------------------------------------
Objective value: 112148.26 EUR
Expected Profit: 112148.26 EUR
CVaR: 0.0000000000 EUR
VaR: 0.00 EUR
Eta values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
# Indices of out-of-sample scenarios
out_of_sample_indices = list(set(all_indices) - set(in_sample_scenarios))

# Sample 950 out-of-sample scenarios without replacement
out_sample_indices = random.sample(out_of_sample_indices, 950)
print(out_sample_indices)

# Extract out-of-sample scenarios from the dictionary containing all scenarios
out_sample_scenarios = {}
for i, idx in enumerate(out_sample_indices):
    out_sample_scenarios[str(i)] = all_scenarios[str(idx)]
    out_sample_scenarios[str(i)]['Original Index'] = idx

# Print information
print('Number of out-of-sample scenarios:', len(out_sample_scenarios))


In [3]:
alpha = 0.9
beta_values = np.arange(0,1 + 0.1, 0.1)

In [10]:
# optimal_DA_bids = pd.read_csv('Data/optimal_DA_bids.csv', index_col=0)

# optimal_DA_oneprice = optimal_DA_bids['One-price Bids [MW]'].values
# optimal_DA_twoprice = optimal_DA_bids['Two-price Bids [MW]'].values


In [8]:
def analyze_in_sample_op(num_in_sample_scenarios, all_scenarios, WIND_CAPACITY, T, alpha, beta_values, out_sample_scenarios):
    OMEGA = num_in_sample_scenarios

    # Sample in-sample scenarios without replacement
    in_sample_scenarios = random.sample(range(len(all_scenarios.keys()) - 1), num_in_sample_scenarios)

    scenarios = {}
    for i, idx in enumerate(in_sample_scenarios):
        scenarios[str(i)] = all_scenarios[str(idx)]
        scenarios[str(i)]['Original Index'] = idx

    # Run optimization for one-price
    results_per_beta_op, p_DA_values_per_beta_op = cvr_op_scheme(scenarios, WIND_CAPACITY, T, OMEGA, alpha, beta_values, minimize_printouts=False, mip_gap = 1e-4) 
    beta_val = 0.5
    optimal_DA_oneprice = np.array(p_DA_values_per_beta_op[beta_val])
    expected_profit = results_per_beta_op[beta_val]['Expected Profit']
    
    # Calculate earnings for in-sample scenarios
    in_sample_profits = 0
    for scenario_idx in in_sample_scenarios:
        scenario = all_scenarios[str(scenario_idx)]
        x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_oneprice
        in_sample_profits += np.sum(x)
    
    # Calculate payments or earnings for out-of-sample scenarios
    out_sample_profits = 0
    for scenario_idx in out_sample_scenarios:
        scenario = out_sample_scenarios[scenario_idx]
        x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_oneprice
        realized_imbalance = np.array(scenario['Wind Power [MW]']) - optimal_DA_oneprice
        price_coefficient = 1.2 * np.array(scenario['System Balance State']) + 0.9 * (1 - np.array(scenario['System Balance State']))
        y = realized_imbalance * price_coefficient * scenario['Spot Price [EUR/MWh]']
        payment_earnings = x + y
        out_sample_profits += np.sum(payment_earnings)
        
    # Calculate average profits over in-sample scenarios
    average_in_sample_profits = expected_profit#in_sample_profits / num_in_sample_scenarios

    # Calculate average profits over out-of-sample scenarios
    average_out_sample_profits = out_sample_profits / len(out_sample_scenarios)

    return average_in_sample_profits, average_out_sample_profits

tolerance = 10000
num_iterations = 5
best_num_in_sample_scenarios = None

for i in range(num_iterations):
    num_in_sample_scenarios = random.randint(250, 500)  
    avg_in_sample_profits, avg_out_sample_profits = analyze_in_sample_op(num_in_sample_scenarios, all_scenarios, WIND_CAPACITY, T, alpha, beta_values, out_sample_scenarios)
    print(avg_in_sample_profits,avg_out_sample_profits)
    print(num_in_sample_scenarios)

    # Check if the absolute difference between in-sample and out-of-sample profits is below the tolerance
    if abs(avg_in_sample_profits - avg_out_sample_profits) < tolerance:
        best_num_in_sample_scenarios = num_in_sample_scenarios
        break

# Print the results
if best_num_in_sample_scenarios is not None:
    print(f"Number of in-sample scenarios yielding similar in-sample and out-of-sample profits: {best_num_in_sample_scenarios}")
else:
    print("Could not find a number of in-sample scenarios yielding similar in-sample")

# for i in range(num_iterations):
#     num_in_sample_scenarios = random.randint(200, 400)  
#     avg_earnings_one_price = analyze_in_sample_op(num_in_sample_scenarios, all_scenarios, WIND_CAPACITY, T, alpha, beta_values)
#     in_sample_results[num_in_sample_scenarios] = avg_earnings_one_price
    
#     
#     if i > 0:
#         prev_avg_earnings_one_price = list(in_sample_results.values())[-2]  
#         if abs(avg_earnings_one_price - prev_avg_earnings_one_price) < tolerance:
#             print(f"Convergence achieved after {i+1} iterations.")
#             break

# # Print the results
# print("In-Sample Scenario Count\tAverage Out-of-Sample Earnings")
# for num_in_sample_scenarios, avg_earnings_one_price in in_sample_results.items():
#     print(f"{num_in_sample_scenarios}\t\t\t\t{avg_earnings_one_price}")

Solving for beta = 0.00...
-----------------------------------------------
Objective value: 256222.81 EUR
Expected Profit: 256222.81 EUR
CVaR: -134543820.7801974416 EUR
VaR: -19316.32 EUR
Eta values: [14058073.440408207, 6978184.414902801, 6968205.163356361, 6907649.833488124, 7005532.224811416, 7045771.814580197, 7076903.441399032, 6955121.823196222, 14115385.366915349, 7096864.027005537, 14171541.064289482, 13896119.30920715, 14004271.466521546, 14026695.656129638, 7016910.792710639, 6871023.933018664, 6922630.86209031, 7052159.464200965, 7021625.125758668, 13910874.464213705, 6955643.393915769, 7000522.058906873, 6845793.35242106, 14003909.553997535, 6986144.5586668225, 6999050.421458068, 7007994.8697098335, 13950993.361090004, 13905386.284972971, 14162025.588062331, 13972654.733574845, 13979921.560359234, 7001547.525563288, 6964641.694108695, 7014018.958933034, 7020083.193522356, 13973012.7784645, 13988014.549216146, 13947115.246397331, 14212907.3652124, 14000188.827559303, 6985345

In [6]:
def analyze_in_sample_tp(num_in_sample_scenarios, all_scenarios, WIND_CAPACITY, T, alpha, beta_values, out_sample_scenarios):
    
    OMEGA = num_in_sample_scenarios

    in_sample_scenarios = random.sample(range(len(all_scenarios.keys()) - 1), num_in_sample_scenarios)

    # Extract sampled in-sample scenarios from dictionary containing all scenarios
    scenarios = {}
    for i, idx in enumerate(in_sample_scenarios):
        scenarios[str(i)] = all_scenarios[str(idx)]
        scenarios[str(i)]['Original Index'] = idx

    # Run optimization for two-price scheme
    results_per_beta_tp, p_DA_values_per_beta_tp = cvr_tp_scheme(scenarios, WIND_CAPACITY, T, OMEGA, alpha, beta_values, minimize_printouts=False, mip_gap = 1e-4) 
    beta_val = 0.5
    optimal_DA_twoprice = np.array(p_DA_values_per_beta_tp[beta_val])

    # Calculate earnings for in-sample scenarios
    in_sample_profits = 0
    for scenario_idx in in_sample_scenarios:
        scenario = all_scenarios[str(scenario_idx)]
        x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_twoprice
        in_sample_profits += np.sum(x)
    
    # Calculate payments or earnings for out-of-sample scenarios
    out_sample_profits = 0
    for scenario_idx in out_sample_scenarios:
        scenario = out_sample_scenarios[scenario_idx]

        x = scenario['Spot Price [EUR/MWh]'] * optimal_DA_twoprice

        realized_imbalance = np.array(scenario['Wind Power [MW]']) - optimal_DA_twoprice

        price_coefficients = np.zeros(T)

        for t in range(T):
            if (realized_imbalance[t] > 0) and (scenario['System Balance State'][t] == 0):
                price_coefficients[t] = 0.9
            elif (realized_imbalance[t] > 0) and (scenario['System Balance State'][t] == 1):
                price_coefficients[t] = 1.0
            elif (realized_imbalance[t] < 0) and (scenario['System Balance State'][t] == 0):
                price_coefficients[t] = -1.0
            elif (realized_imbalance[t] < 0) and (scenario['System Balance State'][t] == 1):
                price_coefficients[t] = -1.2

        y = np.abs(realized_imbalance) * price_coefficients * scenario['Spot Price [EUR/MWh]']

        payment_earnings = x + y

        out_sample_profits += np.sum(payment_earnings)

    # Calculate average profits over in-sample scenarios
    average_in_sample_profits = in_sample_profits / num_in_sample_scenarios

    # Calculate average profits over out-of-sample scenarios
    average_out_sample_profits = out_sample_profits / len(out_sample_scenarios)

    return average_in_sample_profits, average_out_sample_profits

tolerance = 600
num_iterations = 5
best_num_in_sample_scenarios = None

for i in range(num_iterations):
    num_in_sample_scenarios = random.randint(200, 500)  
    avg_in_sample_profits, avg_out_sample_profits = analyze_in_sample_tp(num_in_sample_scenarios, all_scenarios, WIND_CAPACITY, T, alpha, beta_values, out_sample_scenarios)
    print(avg_in_sample_profits, avg_out_sample_profits)

    # Check if the absolute difference between in-sample and out-of-sample profits is below the tolerance
    if abs(avg_in_sample_profits - avg_out_sample_profits) < tolerance:
        best_num_in_sample_scenarios = num_in_sample_scenarios
        break

# Print the results
if best_num_in_sample_scenarios is not None:
    print(f"Number of in-sample scenarios yielding similar in-sample and out-of-sample profits: {best_num_in_sample_scenarios}")
else:
    print("Could not find a number of in-sample scenarios yielding similar in-sample")



# tolerance = 0.05 
# num_iterations = 10
# in_sample_results = {}

# for i in range(num_iterations):
#     num_in_sample_scenarios = random.randint(200, 400)  
#     avg_earnings_two_price = analyze_in_sample_tp(num_in_sample_scenarios, all_scenarios, WIND_CAPACITY, T, alpha, beta_values)
#     in_sample_results[num_in_sample_scenarios] = avg_earnings_two_price
    

#     if i > 0:
#         prev_avg_earnings_two_price = list(in_sample_results.values())[-2]  
#         if abs(avg_earnings_two_price - prev_avg_earnings_two_price) < tolerance:
#             print(f"Convergence achieved after {i+1} iterations.")
#             break

# # Print the results
# print("In-Sample Scenario Count\tAverage Out-of-Sample Earnings")
# for num_in_sample_scenarios, avg_earnings_two_price in in_sample_results.items():
#     print(f"{num_in_sample_scenarios}\t\t\t\t{avg_earnings_two_price}")


Solving for beta = 0.00...
-----------------------------------------------
Objective value: 229307.59 EUR
Expected Profit: 229307.59 EUR
CVaR: 0.0000000000 EUR
VaR: 0.00 EUR
Eta values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0